<a href="https://colab.research.google.com/github/yashrohilla25/Googleimagedown/blob/main/googleimage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install python-dotenv


In [ ]:
SENDER_EMAIL=rohillayash251203@gmail.com
SENDER_PASSWORD=your_app_password
API_KEY=your_google_api_key
CX=your_google_cx


In [ ]:
.env


In [3]:
import os
import requests
import zipfile
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders
import shutil
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()

def search_and_download_images(query, num_images, api_key, cx, download_folder='images'):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        'q': query,
        'num': 10,  # Max 10 images per request
        'imgSize': 'medium',
        'searchType': 'image',
        'key': api_key,
        'cx': cx,
        'fileType': 'jpg|png|gif',
    }

    downloaded_images = []
    start_index = 1

    while len(downloaded_images) < num_images and start_index <= 91:
        params['start'] = start_index
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            results = response.json()

            if 'items' not in results:
                break

            for item in results['items']:
                if len(downloaded_images) >= num_images:
                    break

                image_url = item['link']
                try:
                    img_response = requests.get(image_url, timeout=10)
                    img_response.raise_for_status()

                    file_extension = os.path.splitext(image_url)[-1].lower()
                    if file_extension not in ['.jpg', '.jpeg', '.png', '.gif']:
                        file_extension = '.jpg'

                    image_path = f"{download_folder}/image_{len(downloaded_images)+1}{file_extension}"
                    with open(image_path, 'wb') as f:
                        f.write(img_response.content)
                    downloaded_images.append(image_path)
                except requests.RequestException as e:
                    print(f"Error downloading image: {e}")

            start_index += 10
        except requests.RequestException as e:
            print(f"Error in API request: {e}")
            break

    return downloaded_images

def compress_images_to_zip(image_paths, zip_name='images.zip'):
    with zipfile.ZipFile(zip_name, 'w') as zipf:
        for image_path in image_paths:
            zipf.write(image_path, os.path.basename(image_path))
    return zip_name

def send_email_with_attachment(receiver_email, subject, body, attachment_path, sender_email, sender_password):
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'plain'))

    with open(attachment_path, "rb") as attachment:
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(attachment.read())

    encoders.encode_base64(part)
    part.add_header('Content-Disposition', f"attachment; filename= {os.path.basename(attachment_path)}")
    msg.attach(part)

    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, receiver_email, msg.as_string())
        print("Email sent successfully!")
    except smtplib.SMTPAuthenticationError:
        print("Failed to authenticate. Please check your email and App Password.")
    except Exception as e:
        print(f"Failed to send email: {e}")

# Main execution
def main():
    # Load sensitive information from environment variables
    sender_email = os.getenv('SENDER_EMAIL')
    sender_password = os.getenv('SENDER_PASSWORD')
    api_key = os.getenv('API_KEY')
    cx = os.getenv('CX')

    if not all([sender_email, sender_password, api_key, cx]):
        print("Error: Missing environment variables. Please set them in a .env file.")
        return

    receiver_email = input("Enter the receiver's email address: ")
    query = input("Enter the search query for images: ")
    num_images = int(input("Enter the number of images to download (max 100): "))

    # Download images
    downloaded_images = search_and_download_images(query, num_images, api_key, cx)

    if len(downloaded_images) < num_images:
        print(f"Only {len(downloaded_images)} images were downloaded.")

    # Compress images to a zip file
    zip_file = compress_images_to_zip(downloaded_images)

    # Send email with the zip file attached
    send_email_with_attachment(
        receiver_email=receiver_email,
        subject=f'Your requested images for "{query}"',
        body=f'Here are the {len(downloaded_images)} images you requested for the search query: "{query}"',
        attachment_path=zip_file,
        sender_email=sender_email,
        sender_password=sender_password
    )

    # Clean up: Remove downloaded files and zip
    os.remove(zip_file)
    for file in downloaded_images:
        os.remove(file)
    shutil.rmtree(os.path.dirname(downloaded_images[0]))

if __name__ == "__main__":
    main()


Error: Missing environment variables. Please set them in a .env file.
